## Defining Bug Types for commitpackft dataset

*Approach 1*

Description: Δημιουργία ενός λεξιλογίου με λέξεις και φράσεις κλειδιά, βάση του οποίου θα γίνει η κατηγοριοποιήση των δειγμάτων. Προγραμματιστικά, θα ελέγχεται αν υπάρχουν φράσεις κλειδιά από κάθε τύπο σφάλματος στο commit message του κάθε δείγματος (μπορεί να εμπεριέχονται φράσεις για παραπάνω απο έναν τύπο σφάλματος). Η κατηγοριοποιήση των commit μυνημάτων σε τύπους σφαλμάτων γίνεται ανέμασα σε πέντε κλάσεις (general, functionality, performance/compatibility, network/security, ui-ux, mobile) οι οποίες αντιστοιχίζονται με λέξεις κλειδιά, και κάθε δείγμα παίρνει μία η παραπάνω κλάσεις όταν εμπερειέχεται κάποια λέξη κλειδί από ένα συγκεκριμένο τύπο στπ commit μύνημα.

Cons: 
- Η λιστα με τα keywords για καθε τυπο μπορει να επεκταθει

In [1]:
from modules.filters import identify

identify(query="select * from commitpackft", sqlite_path='commitpack-datasets.db', psql_convert=True)

# Create Word Count Dictionary
# Processing Messages
# Detecting bugs
# Classifying samples


In [2]:
import sqlite3
import pandas as pd
from modules.filters import split

con = sqlite3.connect('commitpack-datasets.db')
df = pd.read_sql_query("select * from commitpackft_bugs", con).set_index('index')
bTypes = pd.read_sql_query('select * from bug_types', con).set_index('index')
bTypes['keywords'] = bTypes['keywords'].apply(lambda kws: kws.split(','))
bTypes = bTypes.to_dict(orient='records')

sample = df.iloc[100].to_dict()
for bT in bTypes:
    if any(word in sample['processed_message'] for word in bT['keywords'])

{'type': 'compatibility-performance', 'keywords': ['compatib', 'browser', 'platform', 'device', 'support', 'version', 'resolution', 'available', 'backward', 'npm', 'node', 'react', 'vue', 'angular', 'jquery', 'ES', 'lint', 'webpack', 'typescript', 'depencency', 'size', 'payload', 'mb', 'test', 'async', 'sync', 'optimiz', 'runtime', 'load', 'callback', 'json', 'post', 'for', 'each', 'loop', 'chrome', 'opera', 'edge', 'firefox', 'safari']} fix final safari firefox


In [2]:
import sqlite3
import pandas as pd
from modules.filters import split

con = sqlite3.connect('commitpack-datasets.db')
df = pd.read_sql_query("select * from commitpackft_classified", con).set_index('index')
bTypes = pd.read_sql_query('select * from bug_types', con).set_index('index')
bTypes['keywords'] = bTypes['keywords'].apply(lambda kws: kws.split(','))
bTypes = bTypes['type'].tolist()
bTypes.append('general')

dists = {}
for type in bTypes:
    dists[type] = 0
for _, row in df.iterrows():
    rowD = row.to_dict()
    for type in bTypes:
        if rowD['bug_type'] == type:
            dists[type] += 1
dists

{'mobile': 24,
 'functionality': 2409,
 'ui-ux': 659,
 'compatibility-performance': 2156,
 'network-security': 728,
 'general': 5530}